In [154]:
#Testing formulation for multiple monitoring (log) and interdiction
#Single interdiction

using JuMP
using Gurobi


global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
global paths = [[1 2 3 5],
                [1 2 5],
                [1 3 5],
                [1 4 5]] 

global N = collect(1:maximum(arcs))
global d = [4,5,10,10,4, 6, 9]
global M = [[1,4], [4,7]]
# global M = [[1], [4]]
global b = 10


global sizeM = length(M)
global origin = 1
global destination = 5 
global numArcs = length(arcs[:,1])
global numPaths = length(paths)
global p = [0.3, 0.4, 0.5, 0.6, 0.7, 0.6, 0.5]
global logP = []
for i = 1:numArcs
   push!(logP, -log(1-p[i])) 
end

println(logP)

gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
MP = Model(() -> Gurobi.Optimizer(gurobi_env))
cons = Array{JuMP.ConstraintRef}(undef, numPaths)
@variable(MP, logX[1:numArcs], Bin)
@variable(MP, u >= -(10^6))
@variable(MP, logY[1:numArcs] <= 0)
# @variable(MP, P[1:numPaths])
@objective(MP, Min, u)

for l = 1:numPaths
    myPath = paths[l]
    pathL = []
    for a = 1:length(myPath)-1
        i = myPath[a]
        j = myPath[a+1]
        temp = findall(arcs[:,1].==i)
        for _ind in temp
            if j == arcs[_ind,2]
               push!(pathL, _ind) 
            end
        end
    end
    cons[l] = @constraint(MP, u >= sum(logP[a] + logY[a] for a in pathL))
end

# for n in N
#     if n != 1 && n!=5
#         incoming = findall(arcs[:,2].==n)
#         outgoing = findall(arcs[:,1].==n)
#         @constraint(MP, sum(logY[a] for a in outgoing) - sum(logY[a] for a in incoming) == 0)
#     end
#     if n == 1
#         incoming = findall(arcs[:,2].==n)
#         outgoing = findall(arcs[:,1].==n)
#         @constraint(MP, sum(logY[a] for a in outgoing) - sum(logY[a] for a in incoming) == 0)
#     end
# end
# s
logP_feas = Any[0.356675, -0.0, -0.0, 0.916291, -0.0, -0.0, -0.0]
arc_feas = findall(logP_feas.>0)
println("arc_feas = ", arc_feas)
@constraint(MP, sum(logY[a] for a in arc_feas) >= -sum(logP_feas[a] for a = 1:numArcs) )
# logP_feas = Any[0.693147, -0.0, -0.0, 0.693147, -0.0, -0.0, -0.0]
# for a = 1:numArcs
#     @constraint(MP, logY[a] >= -logP_feas[a])
# end
# println(logP_feas)
# for l = 1:numPaths
#     myPath = paths[l]
#     pathL = []
#     for a = 1:length(myPath)-1
#         i = myPath[a]
#         j = myPath[a+1]
#         temp = findall(arcs[:,1].==i)
#         for _ind in temp
#             if j == arcs[_ind,2]
#                push!(pathL, _ind) 
#             end
#         end
#     end
#     @constraint(MP, u >= sum(logP_feas[a] + logY[a] for a in pathL))
# end

# @constraint(MP, sum(d[i]*x[i] for i=1:numArcs) <= b)

optimize!(MP)
println(MP)
# x_now = JuMP.value.(logX)#zeros(arcNum)
# println(x_now)
y_now = JuMP.value.(logY)
println(y_now)
u_now = JuMP.value.(u)
# println(u_now)
# println(value(cons[1]))
println("Obj = ", JuMP.objective_value.(MP))



Any[0.356675, 0.510826, 0.693147, 0.916291, 1.20397, 0.916291, 0.693147]
arc_feas = [1, 4]
Min u
Subject to
 u - logY[1] - logY[4] - logY[6] >= 2.1892564076870427
 u - logY[1] - logY[5] >= 1.5606477482646683
 u - logY[2] - logY[6] >= 1.4271163556401456
 u - logY[3] - logY[7] >= 1.3862943611198906
 logY[1] + logY[4] >= -1.272966
 u >= -1.0e6
 logY[1] <= 0.0
 logY[2] <= 0.0
 logY[3] <= 0.0
 logY[4] <= 0.0
 logY[5] <= 0.0
 logY[6] <= 0.0
 logY[7] <= 0.0
 logX[1] binary
 logX[2] binary
 logX[3] binary
 logX[4] binary
 logX[5] binary
 logX[6] binary
 logX[7] binary

[0.0, 0.0, -1.0e6, 0.0, -1.0e6, -1.0e6, 0.0]
Obj = -1.0e6


In [156]:
global A_M = [1 0 0 1 0 0 0;
           0 0 0 1 0 0 1]
function Feasibility(logY)
    println(logY)
    global arcs, paths,N, d, M,A_M, b, p,logP,sizeM, origin,destination,numArcs, numPaths 
    RHS = []
    for a = 1:numArcs
        push!(RHS, 1-exp(logY[a]))
    end
    println("RHS = ", RHS)
    
    gurobi_env = Gurobi.Env()
    setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
    SP = Model(() -> Gurobi.Optimizer(gurobi_env))

    @variable(SP, x[1:numArcs], Bin)
    @variable(SP, y[1:sizeM,1:numArcs], Bin)
    @variable(SP, s[1:numArcs] >=0)

    @objective(SP, Min, sum(s[a] for a = 1:numArcs) )
    
#     @constraint(SP, sum(d[a]*x[a] for a = 1:numArcs) <= b)
    for m = 1:sizeM
        @constraint(SP, sum(d[a]*y[m,a] for a = 1:numArcs) <= b)
        @constraint(SP, sum(y[m,a] for a = 1:numArcs) >= 1)
    end
    
    #Monitoring decision:
    #[[1,4], [4,7]]
    for a = 1:numArcs
#         for m = 1:sizeM
            @constraint(SP, sum(A_M[m,a]*y[m,a] for m = 1:sizeM) +s[a] >= RHS[a]) #exp(logY[a]))
#         end
    end
    println(SP)
    optimize!(SP)
    y_now = JuMP.value.(y)
    println("y = ", y_now)
    s_now = JuMP.value.(s)
    println("s = ", s_now)
    println("Obj = ", JuMP.objective_value.(SP))
    return y_now, s_now
end
# y_now = 
y, s = Feasibility(y_now)


[0.0, 0.0, -1.0e6, 0.0, -1.0e6, -1.0e6, 0.0]
RHS = Any[0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]
Min s[1] + s[2] + s[3] + s[4] + s[5] + s[6] + s[7]
Subject to
 y[1,1] + y[1,2] + y[1,3] + y[1,4] + y[1,5] + y[1,6] + y[1,7] >= 1.0
 y[2,1] + y[2,2] + y[2,3] + y[2,4] + y[2,5] + y[2,6] + y[2,7] >= 1.0
 y[1,1] + s[1] >= 0.0
 s[2] >= 0.0
 s[3] >= 1.0
 y[1,4] + y[2,4] + s[4] >= 0.0
 s[5] >= 1.0
 s[6] >= 1.0
 y[2,7] + s[7] >= 0.0
 4 y[1,1] + 5 y[1,2] + 10 y[1,3] + 10 y[1,4] + 4 y[1,5] + 6 y[1,6] + 9 y[1,7] <= 10.0
 4 y[2,1] + 5 y[2,2] + 10 y[2,3] + 10 y[2,4] + 4 y[2,5] + 6 y[2,6] + 9 y[2,7] <= 10.0
 s[1] >= 0.0
 s[2] >= 0.0
 s[3] >= 0.0
 s[4] >= 0.0
 s[5] >= 0.0
 s[6] >= 0.0
 s[7] >= 0.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 y[1,1] binary
 y[2,1] binary
 y[1,2] binary
 y[2,2] binary
 y[1,3] binary
 y[2,3] binary
 y[1,4] binary
 y[2,4] binary
 y[1,5] binary
 y[2,5] binary
 y[1,6] binary
 y[2,6] binary
 y[1,7] binary
 y[2,7] binary

y = [0.0 0.0 0.0 0

([0.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0], [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0])

In [157]:
y_strategy = [0.5 0.5]
# println(y)
# p_new = (sum(A_M.*y[1,m] for m = 1:sizeM)#).*p'

A_M[1,:] = A_M[1,:].*y[1,:] #*0.5
A_M[2,:] = A_M[2,:].*y[2,:] 
A_M = A_M*0.5
p_new = [1 1]*A_M

1×7 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [138]:
logP_feas = []
for i = 1:numArcs
   push!(logP_feas, -log(1-p_new[i])) 
end

In [139]:
println("logP_feas = ",logP_feas)

logP_feas = Any[0.693147, -0.0, -0.0, 0.693147, -0.0, -0.0, -0.0]
